In [1]:
import pandas as pd
from math import sqrt
from scipy.stats import pearsonr
from sklearn.preprocessing import scale
from sklearn.linear_model import Lasso, ElasticNet, Ridge
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import cross_validate, KFold

## Part I: Training a classifier based on the PMEmo dataset
### Load the training data
The raw dataset consists of 794 songs × 6374 features, including 767 labeled songs.  
Select 260 core features to reduce the computing complexity.  
Core features: the average and the standard deviation of the 65 LLDs as well as their first-order derivatives

In [25]:
def load_data_from_csv(feature_path, annotation_path):
    feature = pd.read_csv(feature_path)
    annotation = pd.read_csv(annotation_path)
    data = pd.merge(feature, annotation)
    col = [0] * 260
    for i in range(118):
        col[i*2] = 16 + i*31
        col[i*2+1] = 19 + i*31
        
    return scale(data.iloc[:, col].values), data.loc[:, 'Arousal(std)'].values, data.loc[:, 'Valence(std)'].values

In [26]:
feature_path = "./training data/static_features.csv"
annotation_path = "./training data/static_annotations_std.csv"
X, y_a, y_v = load_data_from_csv(feature_path, annotation_path)

audspec_lengthL1norm_sma_meanSegLen
audspec_lengthL1norm_sma_segLenStddev
audspecRasta_lengthL1norm_sma_meanSegLen
audspecRasta_lengthL1norm_sma_segLenStddev
pcm_RMSenergy_sma_meanSegLen
pcm_RMSenergy_sma_segLenStddev
pcm_zcr_sma_meanSegLen
pcm_zcr_sma_segLenStddev
audspec_lengthL1norm_sma_de_meanSegLen
audspec_lengthL1norm_sma_de_segLenStddev
audspecRasta_lengthL1norm_sma_de_meanSegLen
audspecRasta_lengthL1norm_sma_de_segLenStddev
pcm_RMSenergy_sma_de_meanSegLen
pcm_RMSenergy_sma_de_segLenStddev
pcm_zcr_sma_de_meanSegLen
pcm_zcr_sma_de_segLenStddev
audSpec_Rfilt_sma[0]_meanSegLen
audSpec_Rfilt_sma[0]_segLenStddev
audSpec_Rfilt_sma[1]_meanSegLen
audSpec_Rfilt_sma[1]_segLenStddev
audSpec_Rfilt_sma[2]_meanSegLen
audSpec_Rfilt_sma[2]_segLenStddev
audSpec_Rfilt_sma[3]_meanSegLen
audSpec_Rfilt_sma[3]_segLenStddev
audSpec_Rfilt_sma[4]_meanSegLen
audSpec_Rfilt_sma[4]_segLenStddev
audSpec_Rfilt_sma[5]_meanSegLen
audSpec_Rfilt_sma[5]_segLenStddev
audSpec_Rfilt_sma[6]_meanSegLen
audSpec_Rfilt_sm

### Setup Metric & Regression Classifiers
Metric: (1) Root Mean Square Error(RMSE); (2) Pearson Correlation Coefficient(PCC)  
Regressor: Lasso, Ridge, Elastic Net, SVR(kernel = rbf/linear/poly)

In [4]:
def rmse(y, y_pred):
    return sqrt(mean_squared_error(y, y_pred))

def pcc(y, y_pred):
    return pearsonr(y, y_pred)[0]

Regressors = {
    'Lasso': Lasso(),
    'Ridge': Ridge(),
    'ElasticNet': ElasticNet(),
    'SVRrbf': SVR(kernel='rbf', gamma='scale'),
    'SVRlinear': SVR(kernel='linear', gamma='scale'),
    'SVRpoly': SVR(kernel='poly', gamma='scale'),
    'DT': DecisionTreeRegressor(max_depth=5),
    'RF': RandomForestRegressor(max_depth=5, n_estimators=10, max_features=1)
}

### Training Classifiers & Evaluation

In [5]:
def cross_val_regression(regressors, features, labels):
    columns = list(Regressors.keys())
    res = pd.DataFrame(columns=columns)
    scorer = {'rmse': make_scorer(rmse)}
    
    for name, reg in Regressors.items():
        reg_score = cross_validate(reg, 
                                   features, 
                                   labels, 
                                   scoring = scorer,
                                   cv = 10, 
                                   return_train_score = False,
                                   error_score = 'raise') 
        res.loc['RMSE', name] = round(reg_score['test_rmse'].mean(), 3)
        #res.loc['PCC', name] = reg_score['test_pcc'].mean()
    return res

In [7]:
scores_Arousal = cross_val_regression(Regressors, X, y_a)
scores_Arousal

Lasso  Ridge ElasticNet SVRrbf SVRlinear
RMSE  0.184  0.216      0.184  0.146     0.226

In [9]:
scores_Valence = cross_val_regression(Regressors, X, y_v)
scores_Valence

Lasso  Ridge ElasticNet SVRrbf SVRlinear
RMSE  0.162  0.196      0.162  0.141     0.209

## Part II: Recognize emotions from new music segments
### Load the testing data

In [ ]:
def load_data_from_txt(feature_path, annotation_path):
    feature = pd.read_csv(feature_path)
    annotation = pd.read_csv(annotation_path)
    data = pd.merge(feature, annotation)
    col = [0] * 260
    for i in range(118):
        col[i*2] = 16 + i*31
        col[i*2+1] = 19 + i*31
        
    return scale(data.iloc[:, col].values), data.loc[:, 'Arousal(std)'].values, data.loc[:, 'Valence(std)'].values

In [ ]:
feature_path = "./training data/static_features.txt"
annotation_path = "./training data/static_annotations_std.csv"
X, y_a, y_v = load_data_from_txt(feature_path, annotation_path)